In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [ ]:
%load_ext google.cloud.bigquery

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


##Pertanyaan 1

Grouping age apa dan gender apa yg memiliki transaksi paling tinggi di BeeCycle?

In [ ]:
%%bigquery --project avid-compound-391908


with grup_usia as (
select
customer_id,
gender,
birthdate,
case
  when date_diff(current_date, birthdate, year) <= 20 then 'grup u-20'
  when date_diff(current_date, birthdate, year) between 21 and 40 then 'grup 21-40'
  when date_diff(current_date, birthdate, year) between 41 and 60 then 'grup 41-60'
  else 'grup di atas 60'
end as kelompok_usia
from `dibimbing.dim_customer`)

select
grup_usia.gender,
grup_usia.kelompok_usia,
sum(fact_sales.totalprice_rupiah) as total_sales
from
dibimbing.fact_sales inner join grup_usia
on fact_sales.customer_id = grup_usia.customer_id
group by 1,2
order by 3 desc


Query is running:   0%|          |

Downloading:   0%|          |

,gender,kelompok_usia,total_sales
0,F,grup 41-60,1.974341e+10
1,F,grup 21-40,1.940950e+10
2,M,grup 21-40,1.861962e+10
3,M,grup 41-60,1.689372e+10
4,M,grup di atas 60,1.354304e+09
5,F,grup di atas 60,1.158323e+09


grup age dan gender yang memiliki transaksi paling besar adalah kelompok usia 41-60 tahun dan gendernya female

##Pertanyaan 2

Warna apa di tiap tahun yg menjadi warna paling populer dibeli oleh customer?

In [ ]:
%%bigquery --project avid-compound-391908

with tahun as (
select
  product_id,
  extract(year FROM order_date)as year
from
  dibimbing.fact_sales )

select
tahun.year,
dim_product.color,
(count(dim_product.color)) as jumlah
from
  tahun
left join
  dibimbing.dim_product
on
  tahun.product_id = dim_product.product_id
where color != 'NA'
group by 1,2
order by 1,3 desc


Query is running:   0%|          |

Downloading:   0%|          |

,year,color,jumlah
0,2016,Red,216
1,2016,Black,64
2,2016,Silver,59
3,2017,Red,223
4,2017,Black,203
5,2017,Silver,125
6,2017,Yellow,10
7,2018,Black,503
8,2018,Silver,351
9,2018,Blue,230


warna populer di setiap tahun

2016 : red 216 penjualan

2017 : red 223 penjualan

2018 : black 503 penjualan

2019 : black 281 penjualan

##Pertanyaan 3

TOP 10 product name apa saja yg paling populer dari masing-masing territory>

In [ ]:
%%bigquery --project avid-compound-391908
with kotak as (
select
region,
fact_sales.territory_id,
product_id,
count(fact_sales.product_id) as jumlah
from dibimbing.fact_sales
left join dibimbing.dim_territory
on fact_sales.territory_id = dim_territory.territory_id
group by 1,2,3)


select*from
(select
region,
product_name,
kotak.jumlah,
ROW_NUMBER() OVER (PARTITION BY region Order by kotak.jumlah DESC) AS urutan
from kotak left join dibimbing.dim_product
on kotak.product_id = dim_product.product_id)
where urutan <=10
limit 20

Query is running:   0%|          |

Downloading:   0%|          |

,region,product_name,jumlah,urutan
0,United Kingdom,Water Bottle - 30 oz.,64,1
1,United Kingdom,"Sport-100 Helmet, Blue",42,2
2,United Kingdom,"Sport-100 Helmet, Red",41,3
3,United Kingdom,AWC Logo Cap,41,4
4,United Kingdom,Mountain Bottle Cage,38,5
5,United Kingdom,Road Bottle Cage,36,6
6,United Kingdom,"Sport-100 Helmet, Black",23,7
7,United Kingdom,Touring Tire,20,8
8,United Kingdom,"Mountain-200 Black, 42",19,9
9,United Kingdom,Patch Kit/8 Patches,18,10


## Pertanyaan 4

objective : sub category product

pertanyaanya : sub category apa yang paling banyak dibeli customer dari masing-masing category, berdasarkan rentang usia customer

21-40 tahun

41-60 tahun

di atas 60 tahun

In [ ]:
%%bigquery --project avid-compound-391908

with grup_usia as (
select
customer_id,
geography_id,
gender,
birthdate,
case
  when date_diff(current_date, birthdate, year) <= 20 then 'grup u-20'
  when date_diff(current_date, birthdate, year) between 21 and 40 then 'grup 21-40'
  when date_diff(current_date, birthdate, year) between 41 and 60 then 'grup 41-60'
  else 'grup di atas 60'
end as kelompok_usia
from `dibimbing.dim_customer`)


select
grup_usia.kelompok_usia,
category,
sub_category,
count(sub_category)  as jumlah
from
dibimbing.fact_sales inner join grup_usia
on fact_sales.customer_id = grup_usia.customer_id
inner join dibimbing.dim_product
on dim_product.product_id = fact_sales.product_id
group by 1,2,3
order by 1,2,4 desc



Query is running:   0%|          |

Downloading:   0%|          |

,kelompok_usia,category,sub_category,jumlah
0,grup 21-40,Accessories,Tires and Tubes,553
1,grup 21-40,Accessories,Bottles and Cages,339
2,grup 21-40,Accessories,Helmets,268
3,grup 21-40,Accessories,Fenders,91
4,grup 21-40,Accessories,Hydration Packs,47
5,grup 21-40,Accessories,Cleaners,35
6,grup 21-40,Accessories,Bike Racks,14
7,grup 21-40,Accessories,Bike Stands,10
8,grup 21-40,Bikes,Mountain Bikes,590
9,grup 21-40,Bikes,Road Bikes,299


hasil analisis

tidak ada perbedaan signifikan pada sub_category produk yang paling laku berdasarkan rentang usia. Beberapa sub_category produk menempati pada posisi yang sama di setiap rentang usia. hanya saja perbedaannya,  usia di atas 60 tahun memiliki jumlah belanja yang paling sedikit.

kita bisa fokus pada rentang usia 21 - 40 dan 41-60 tahun. pada rentang usia tersebut adalah rentang usia produktif dalam kata lain suka berkegiatan. berikut ini adalah rekomendasi bisnis yang bisa diambil untuk meningkatkan sales atau mempertahankan sales supaya tidak turun:


1. menempatkan produk-produk unggulan di area toko yang mudah terlihat oleh customer
2. jangan sampai kehabisan stok di dalam toko, pada barang-barang sub_category yang paling laku
3. meingkatkan campaign pada sub_category produk-produk yang paling laku
4. mencari inovasi baru pada top sub_category. supaya customer maskin banyak pilihannya

